In [12]:
# NOTE: you might need to use a different key, here are all the possible options
#for running this first time
#pip install google-api-python-client

from googleapiclient.discovery import build
import json

key = ''

youtube = build('youtube', 'v3', developerKey=key)


In [13]:
#get a list of all the videos within a certain search
n = 500
max_results = 20
total_retrieved = 0
next_token = ''
search_term = 'minecraft+tutorial'
video_ids_list = []

while total_retrieved < n:

  if next_token == '':
    request = youtube.search().list(
    q=search_term,
    part='snippet',
    type='video',
    maxResults=max_results,
    )

  else:
    request = youtube.search().list(
    q=search_term,
    part='snippet',
    type='video',
    maxResults=max_results,
    pageToken=next_token 
    )

  videos_response = request.execute()
  json_clean = json.dumps(videos_response, indent=2)

  #get the individual video ids
  for item in videos_response['items']:
    video_ids_list.append(item['id']['videoId'])

  #set variables to move to the next page
  total_retrieved += 10
  if 'nextPageToken' in videos_response:
   next_token = videos_response['nextPageToken']
  else:
    next_token = ''

print(len(video_ids_list)) 

994


In [14]:
# search through the videos from the list we created

import pandas as pd
# Create a dataframe for the video data
df = pd.DataFrame(columns=['id', 'Video Title', 'Description', 'Publish Date', 'Channel Title', 'Views', 'Likes', 'Dislikes', 'Comments', 'Thumbnail URL', 
                             'Duration', 'Audio Language', 'Closed Captions?', 'Definition', 'Liscened Content'])
df.set_index('id', inplace=True)

for video_id in video_ids_list:
  request = youtube.videos().list(
      part="snippet,contentDetails,statistics",
      id=video_id
  )
  response = request.execute()

  #gather the data for each video
  for item in response['items']:
    publish_date = item['snippet']['publishedAt']
    channel_title = item['snippet']['channelTitle']
    title = item['snippet']['title']
    description = item['snippet']['description']
    duration = item['contentDetails']['duration']
    definition = item['contentDetails']['definition']
    caption = item['contentDetails']['caption']
    licensed_content = item['contentDetails']['licensedContent']

    #handle error where statistics for these metrics are not given in the response
    if 'commentCount' in item['statistics']:
      comment_count = item['statistics']['commentCount']
    else:
      comment_count = None 

    if 'viewCount' in item['statistics']:
      view_count = item['statistics']['viewCount']
    else:
      view_count = None 

    if 'likeCount' in item['statistics']:
      like_count = item['statistics']['likeCount']
    else:
      like_count = None 

    if 'dislikeCount' in item['statistics']:
      dislike_count = item['statistics']['dislikeCount']
    else:
      dislike_count = None   

    if 'defaultAudioLanguage' in item['snippet']:
      audio_language = item['snippet']['defaultAudioLanguage']
    else:
      audio_language = None

    thumbnail_url = thumbnail_url = item['snippet']['thumbnails']['default']['url']
  
    df.loc[video_id] = [title, description, publish_date, channel_title, view_count, like_count, dislike_count, comment_count, thumbnail_url, duration, audio_language, caption, definition, licensed_content]

print(f'Number of records: {len(df)}')
df.to_csv('YouTube.csv')
df.head()

Number of records: 534


,Video Title,Description,Publish Date,Channel Title,Views,Likes,Dislikes,Comments,Thumbnail URL,Duration,Audio Language,Closed Captions?,Definition,Liscened Content
id,,,,,,,,,,,,,,
e-LpPTnme_M,ULTIMATE MINECRAFT STARTER GUIDE! Learn Everyt...,Here is a beginners guide to Minecraft to help...,2019-07-07T21:04:26Z,TheCh0pper,1225121,29192,1424,1194,https://i.ytimg.com/vi/e-LpPTnme_M/default.jpg,PT11M8S,None,false,hd,True
HOKlZxD306o,19 Tips That Could Save Your Life in Minecraft,19 Tips That Could Save Your Life in Minecraft...,2021-09-27T15:33:08Z,Skip the Tutorial,1151818,45941,559,1884,https://i.ytimg.com/vi/HOKlZxD306o/default.jpg,PT10M13S,en,false,hd,True
N2D9TbHDeCY,Minecraft: How to Build a Large Modern House T...,➜Minecraft: How to Make a Modern House - Tuto...,2021-06-30T01:36:16Z,WiederDude,8644035,136763,5675,2775,https://i.ytimg.com/vi/N2D9TbHDeCY/default.jpg,PT26M2S,en-US,false,hd,True
mLSd5yvzL30,Minecraft MODERN House Tutorial,A Minecraft building tutorial of a Modern Hous...,2021-09-28T02:55:31Z,GEVids,51169,6347,119,91,https://i.ytimg.com/vi/mLSd5yvzL30/default.jpg,PT44S,None,false,hd,False
tm6OaV0GPTc,Minecraft Tutorial: How To Make A Modern McDon...,In this tutorial i show you how to make a Mode...,2020-02-29T12:30:59Z,TSMC - Minecraft,3623360,48936,1872,1667,https://i.ytimg.com/vi/tm6OaV0GPTc/default.jpg,PT37M53S,en-GB,false,hd,True
